In [ ]:
import pandas as pd
import os
from datetime import date

def process_file(input_file, output_dir):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(input_file)

    # Perform generic validations
    df1 = df[(df['column1'] > 150) | (df['column2'] > 150)]
    df2 = df[df['column3'] > df['column4']]
    df3 = df[df['column5'].isna()]
    df4 = df[(df['column5'] != 'VALUE') & df['column6'].isna()]
    df5 = df[(df['column7'].isna()) | (df['column7'] == 0)]
    df6 = df[(df['column4'].isna() | (df['column4'] == 0)) & (df['column5'] != 'VALUE')]
    df7 = df[df.duplicated(subset=['column3', 'column8', 'column6'], keep=False)]
    df['column9_numeric'] = pd.to_numeric(df['column9'], errors='coerce')
    df8 = df[df['column9_numeric'] > df['column10']]
    df9 = df[(df['column5'] != 'VALUE') & (df['column11'] == 'VALUE')]
    df10 = df[((df['column12'] == 'N') | df['column12'].isna()) & (df['column13'] == 'Y')]
    df11 = df[((df['column13'] == 'N') | df['column13'].isna()) & (df['column12'] == 'Y')]

    # Combine and find unique and duplicate listings
    all_listings = pd.concat([df1['listing_url'], df2['listing_url'], df3['listing_url'], df4['listing_url'],
                              df5['listing_url'], df6['listing_url'], df7['listing_url'], df8['listing_url'],
                              df9['listing_url'], df10['listing_url'], df11['listing_url']])

    duplicate_listings = all_listings[all_listings.duplicated()]

    # Filter out to retain only duplicates within each validation DataFrame
    df1 = df1[df1['listing_url'].isin(duplicate_listings)]
    df2 = df2[df2['listing_url'].isin(duplicate_listings)]
    df3 = df3[df3['listing_url'].isin(duplicate_listings)]
    df4 = df4[df4['listing_url'].isin(duplicate_listings)]
    df5 = df5[df5['listing_url'].isin(duplicate_listings)]
    df6 = df6[df6['listing_url'].isin(duplicate_listings)]
    df7 = df7[df7['listing_url'].isin(duplicate_listings)]
    df8 = df8[df8['listing_url'].isin(duplicate_listings)]
    df9 = df9[df9['listing_url'].isin(duplicate_listings)]
    df10 = df10[df10['listing_url'].isin(duplicate_listings)]
    df11 = df11[df11['listing_url'].isin(duplicate_listings)]

    # Descriptions for each validation
    descriptions = [
        "Original Data: The original dataset provided.",
        "1. Validation 1: Listings that meet criteria 1.",
        "2. Validation 2: Listings that meet criteria 2.",
        "3. Validation 3: Listings that meet criteria 3.",
        "4. Validation 4: Listings that meet criteria 4.",
        "5. Validation 5: Listings that meet criteria 5.",
        "6. Validation 6: Listings that meet criteria 6.",
        "7. Validation 7: Listings that meet criteria 7.",
        "8. Validation 8: Listings that meet criteria 8.",
        "9. Validation 9: Listings that meet criteria 9.",
        "10. Validation 10: Listings that meet criteria 10.",
        "11. Validation 11: Listings that meet criteria 11."
    ]

    # Creating a dictionary for DataFrames with descriptions
    dfs = {
        'Original Data': df,
        '1. Validation 1': df1,
        '2. Validation 2': df2,
        '3. Validation 3': df3,
        '4. Validation 4': df4,
        '5. Validation 5': df5,
        '6. Validation 6': df6,
        '7. Validation 7': df7,
        '8. Validation 8': df8,
        '9. Validation 9': df9,
        '10. Validation 10': df10,
        '11. Validation 11': df11
    }

    # Output path with today's date
    output_file = os.path.join(output_dir, f'validated_output_{date.today()}.xlsx')

    # Write to Excel with Table of Contents
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        workbook  = writer.book
        worksheet = workbook.add_worksheet('Table of Contents')
        worksheet.write('A1', 'Table of Contents')
        
        row = 2
        for description in descriptions:
            sheet_name, desc = description.split(':', 1)
            sheet_name = sheet_name.strip()
            desc = desc.strip()
            
            worksheet.write(row, 0, sheet_name)
            worksheet.write(row, 1, desc)
            worksheet.write_url(row, 2, f"internal:'{sheet_name}'!A1", string='Go to Sheet')
            row += 1

        for sheet_name, dataframe in dfs.items():
            dataframe.to_excel(writer, index=False, sheet_name=sheet_name)

    print(f"Processed file saved to {output_file}")

# Example function call
input_file = 'your_input_file_path.csv'  # Provide your actual input file path
output_dir = 'your_output_directory'  # Provide your actual output directory path

# Run the processing function
process_file(input_file, output_dir)